In [2]:
import random
import time
from typing import Optional, List, Dict

# =========================================================
# 1. CORE CLASSES (CONSOLIDATED)
# =========================================================

# --- Move Class ---
class Move:
    def __init__(self, name: str, power: int, type: str, category: str = "physical"):
        self.name = name
        self.power = power
        self.type = type
        self.category = category

    def __repr__(self):
        return self.name

# --- Pokemon Classes ---
class PokemonSpecies:
    def __init__(self, name: str, type1: str, base_stats: Dict[str, int]):
        self.name = name
        self.type1 = type1
        self.base_stats = base_stats

class PokemonInstance:
    """Represents an individual Pokémon in battle."""
    def __init__(self, species: PokemonSpecies, level: int = 50):
        self.species = species
        self.level = level
        self.stats = self._calculate_stats()
        self.current_hp = self.stats['hp']
        self.moves: List[Move] = []
        self.ability = self # Simple stub

    def _calculate_stats(self) -> Dict[str, int]:
        # Simple, non-standard stat calculation
        stats = {k: v + self.level * 2 for k, v in self.species.base_stats.items()}
        stats['hp'] = self.species.base_stats['hp'] * 3
        return stats

    def get_stat(self, stat_name: str) -> int:
        return self.stats.get(stat_name.lower(), 0)

    def calculate_damage(self, move: Move, defender: 'PokemonInstance', battle_state: 'BattleState') -> float:
        """Stub damage calculation: Base power + Random variance."""
        base_damage = move.power * (self.level / 100.0)
        damage_variance = random.uniform(0.85, 1.0)
        return base_damage * damage_variance

    def take_damage(self, damage: float):
        self.current_hp = max(0, self.current_hp - int(damage))

    def fainted(self) -> bool:
        return self.current_hp <= 0

    def __str__(self):
        return f"{self.species.name} (Lvl {self.level}) - {self.current_hp}/{self.stats['hp']} HP"

# --- Action Class ---
class Action:
    """Encapsulates a single action chosen by a player or AI."""
    def __init__(self, pokemon: PokemonInstance, move: Move):
        self.pokemon = pokemon
        self.move = move

    def __repr__(self):
        return f"Action: {self.pokemon.species.name} uses {self.move.name}"

# --- BattleState Class ---
class BattleState:
    """Holds the current state of a Pokémon battle."""

    def __init__(self, pokemon1: PokemonInstance, pokemon2: PokemonInstance):
        self.pokemon1 = pokemon1
        self.pokemon2 = pokemon2
        self.turn = 1
        self.weather: Optional[str] = None
        self.field_effects = {}
        self.active_pokemon = [pokemon1, pokemon2]
        self.last_move: Optional[Move] = None

    def next_turn(self):
        """Increment turn counter and apply end-of-turn effects."""
        print(f"\n--- Turn {self.turn} ends ---")
        for p in self.active_pokemon:
            # Placeholder for end-of-turn ability/status checks
            if hasattr(p.ability, "on_end_turn"):
                p.ability.on_end_turn(p)
        self.turn += 1

    def get_turn_order(self) -> List[PokemonInstance]:
        """Return Pokémon in order of who moves first, based on speed."""
        p1_speed = self.pokemon1.get_stat("speed")
        p2_speed = self.pokemon2.get_stat("speed")
        if p1_speed == p2_speed:
            return random.sample(self.active_pokemon, 2)
        return sorted(self.active_pokemon, key=lambda p: p.get_stat("speed"), reverse=True)

    def execute_move(self, attacker: PokemonInstance, defender: PokemonInstance, move: Move):
        """Handles damage, type effectiveness, and applying effects."""

        if attacker.fainted():
            print(f"{attacker.species.name} is unable to move (fainted).")
            return

        print(f"\n{attacker.species.name} used {move.name}!")

        damage = attacker.calculate_damage(move, defender, battle_state=self)
        defender.take_damage(damage)

        self.last_move = move

        print(f"It dealt {int(damage)} damage! {defender}")

        if defender.fainted():
            print(f"!!! {defender.species.name} fainted! !!!")

    def simulate_turn(self, action1: Action, action2: Action):
        """Simulates a full turn using two Action objects."""
        action_map = {
            action1.pokemon: action1,
            action2.pokemon: action2
        }

        order = self.get_turn_order()

        print("\n--- Battle Turn Start ---")

        for pokemon in order:
            action = action_map.get(pokemon)

            # Identify the defender
            defender = self.pokemon2 if pokemon == self.pokemon1 else self.pokemon1

            # Check for win condition mid-turn
            if defender.fainted():
                print(f"{pokemon.species.name} has no target left!")
                break

            # Execute the move
            self.execute_move(pokemon, defender, action.move)

        self.next_turn()

# =========================================================
# 2. GAME LOGIC & HELPERS
# =========================================================

def choose_ai_action(ai_pkmn: PokemonInstance) -> Action:
    """Simple AI: selects a move randomly."""
    # Ensure the AI picks from its available moves
    chosen_move = random.choice(ai_pkmn.moves)
    print(f"AI ({ai_pkmn.species.name}) is ready...")
    return Action(ai_pkmn, chosen_move)

def choose_player_action(player_pkmn: PokemonInstance) -> Action:
    """Mocks player input to keep the simulation simple."""
    print(f"\nYour turn, {player_pkmn.species.name}:")
    for i, move in enumerate(player_pkmn.moves):
        # Displays available moves for player reference
        print(f"[{i+1}] {move.name} (Power: {move.power}, Type: {move.type})")

    # --- MOCK USER INPUT ---
    # For a continuous simulation, we'll choose a random move for the player
    # until you implement real command-line input.
    chosen_move = random.choice(player_pkmn.moves)

    # Optional: If you want to force a specific move for debugging, uncomment this:
    # chosen_move = player_pkmn.moves[0]

    print(f"\n(Player chose {chosen_move.name})")
    return Action(player_pkmn, chosen_move)

def game_loop(battle_state: BattleState):
    """
    The main loop that runs the battle turn by turn until a winner is found.
    """
    player_pkmn = battle_state.pokemon1
    ai_pkmn = battle_state.pokemon2

    while not player_pkmn.fainted() and not ai_pkmn.fainted():
        print("="*40)
        print(f"             TURN {battle_state.turn}             ")
        print("="*40)

        # 1. Get Player Action
        player_action = choose_player_action(player_pkmn)

        # 2. Get AI Action
        ai_action = choose_ai_action(ai_pkmn)

        # 3. Simulate the Turn (updates the battle state and HP)
        battle_state.simulate_turn(player_action, ai_action)

        # Pause briefly to make the output easier to follow
        time.sleep(1)

    # 4. Determine Winner and End Game
    print("\n" + "#"*40)
    if player_pkmn.fainted() and ai_pkmn.fainted():
        print("         IT'S A DRAW! DOUBLE FAINT!         ")
    elif player_pkmn.fainted():
        print(f"         {ai_pkmn.species.name} wins! (Player lost)        ")
    else:
        print(f"         {player_pkmn.species.name} wins! (Player won)         ")
    print("#"*40)


# =========================================================
# 3. INITIALIZATION AND EXECUTION
# =========================================================

if __name__ == "__main__":
    # Define Moves
    tackle = Move(name="Tackle", power=40, type="Normal")
    flamethrower = Move(name="Flamethrower", power=90, type="Fire")
    water_gun = Move(name="Water Gun", power=40, type="Water")
    rock_slide = Move(name="Rock Slide", power=75, type="Rock")

    # Define Species (Charizard is faster, Blastoise has more HP/Defense)
    species_charizard = PokemonSpecies(
        name="Charizard",
        type1="Fire",
        base_stats={"hp": 78, "attack": 84, "defense": 78, "speed": 100}
    )
    species_blastoise = PokemonSpecies(
        name="Blastoise",
        type1="Water",
        base_stats={"hp": 79, "attack": 83, "defense": 100, "speed": 78}
    )

    # Initialize Pokémon
    player_pokemon = PokemonInstance(species_charizard, level=50)
    player_pokemon.moves = [flamethrower, rock_slide] # Player moves

    ai_pokemon = PokemonInstance(species_blastoise, level=50)
    ai_pokemon.moves = [water_gun, tackle] # AI moves

    print("--- BATTLE SETUP ---")
    print(f"Player: {player_pokemon.species.name} (Speed: {player_pokemon.get_stat('speed')})")
    print(f"AI: {ai_pokemon.species.name} (Speed: {ai_pokemon.get_stat('speed')})")
    print(f"HP: {player_pokemon.current_hp} vs {ai_pokemon.current_hp}")
    print("--------------------\n")

    # Initialize Battle State
    battle_state = BattleState(player_pokemon, ai_pokemon)

    # Start the Game Loop
    game_loop(battle_state)

--- BATTLE SETUP ---
Player: Charizard (Speed: 200)
AI: Blastoise (Speed: 178)
HP: 234 vs 237
--------------------

             TURN 1             

Your turn, Charizard:
[1] Flamethrower (Power: 90, Type: Fire)
[2] Rock Slide (Power: 75, Type: Rock)

(Player chose Flamethrower)
AI (Blastoise) is ready...

--- Battle Turn Start ---

Charizard used Flamethrower!
It dealt 43 damage! Blastoise (Lvl 50) - 194/237 HP

Blastoise used Tackle!
It dealt 17 damage! Charizard (Lvl 50) - 217/234 HP

--- Turn 1 ends ---
             TURN 2             

Your turn, Charizard:
[1] Flamethrower (Power: 90, Type: Fire)
[2] Rock Slide (Power: 75, Type: Rock)

(Player chose Flamethrower)
AI (Blastoise) is ready...

--- Battle Turn Start ---

Charizard used Flamethrower!
It dealt 42 damage! Blastoise (Lvl 50) - 152/237 HP

Blastoise used Tackle!
It dealt 19 damage! Charizard (Lvl 50) - 198/234 HP

--- Turn 2 ends ---
             TURN 3             

Your turn, Charizard:
[1] Flamethrower (Power: 90, Typ